In [1]:
import os, sys
from simtk import unit as u
from simtk import openmm as mm
from openmm.app import *
from ctgomartini.api import MartiniTopFile
from ctgomartini.func import read_inputs
import MDAnalysis as mda
import argparse
import datetime


    

    


/home/ys/anaconda3/envs/ctgomartini_test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_electricforce(Ez):
    """
    This method creates force object that applies electric force onto z axis
    of the simulation. The electric force must be in the units of 
    kilojoules_per_mole/nanometer/elementary_charge.
    
    Arguments: 
        Ez as Quantity
    Returns: 
        Force object
    """

    Ez = Ez.in_units_of(u.kilojoules_per_mole/u.nanometer/u.elementary_charge)
    print(Ez)
    potential = "-q*Ez*z"
    force = mm.CustomExternalForce(potential)
    force.addPerParticleParameter("q")
    force.addGlobalParameter("Ez", Ez)

    return force


def apply_electric_permolecule(system, force):
    """
    This method is used to apply electric force to a given system.
    Arguments: 
        system as System
        force as Force
    Returns:
        None
    """
    for f in system.getForces():
        print(f)
    nonbonded = [f for f in system.getForces() if isinstance(f, mm.CustomNonbondedForce)][0]
    print(nonbonded.getParticleParameters(2))
    for i in range(system.getNumParticles()):
        _, charge = nonbonded.getParticleParameters(i)
        #print(_, charge)
        # You have the charge for particle i, so add it to the CustomExternalForce.
        force.addParticle(i, [charge])
    print(i)
    system.addForce(force)
    
def voltage_per_nm_to_openmm(v_per_nm):
    """
    Coverts voltage per nanometer (as volt/nanomer Quantity) to
    kilojoules_per_mole/nanometer/elementary_charge units. 
    
    Arguments: 
        v_per_nm as Quantity
    Returns
        v as Quantity 
    """
    v =  v_per_nm /u.item
    print(v)
    return v.in_units_of(u.kilojoules_per_mole/u.nanometer/u.elementary_charge)



In [3]:
from function import *

def OMM_setSimulation(
        strfile,
        topfile,
        epsilon_r=15.0,
        temperature=310.15,
        double_precision=False):

    # Box_vectors Initiation
    if strfile.split('.')[-1] == 'gro':
        conf = GromacsGroFile(strfile)
        box_vectors = conf.getPeriodicBoxVectors()
    elif strfile.split('.')[-1] == 'pdb':
        conf = PDBFile(strfile)
        box_vectors = conf.getTopology().getPeriodicBoxVectors()
    else:
        raise ValueError(f"Cannot find {strfile}")

    # Set Platform
    if double_precision:
        platform = mm.Platform.getPlatformByName("Reference")
    else:
        platform = mm.Platform.getPlatformByName("CPU")

    # get any defines
    defines = {}
    try:
        with open("defines.txt") as def_file:
            for line in def_file:
                line = line.strip()
                defines[line] = True
    except FileNotFoundError:
        pass

    # Get system and top
    top = MartiniTopFile(
        topfile,
        periodicBoxVectors=box_vectors,
        defines=defines,
    )
    system = top.create_system(
        nonbonded_cutoff=1.1 * u.nanometer,
        epsilon_r=epsilon_r,)
    
    # Add electric fields
    # voltage = -0.02 * u.volt/u.nanometer
    # voltage = voltage_per_nm_to_openmm(voltage)
    # eforce = create_electricforce(voltage)
    # apply_electric_permolecule(system, eforce)

    integrator = mm.LangevinIntegrator(
        temperature * u.kelvin, 1.0 / u.picosecond, 20 * u.femtosecond
    )

    simulation = mm.app.Simulation(top.topology, system, integrator, platform)
    simulation.__dict__['top'] = top
    return simulation


def Compare_OMM_GMX(working_dir, epsilon_r=15.0):
    print(working_dir)
    os.chdir(os.path.join(working_dir, "openmm"))
    strfile = "minimized.gro"
    topfile = "system.top"

    simulation = OMM_setSimulation(strfile, topfile, epsilon_r=epsilon_r, temperature=310.15, double_precision=True)
    OMM_calStrfile(strfile, simulation, set_vsite=True)

    omm_energy=Load_energy(clean=True)
    omm_forces=Load_forces(clean=True)
    print(omm_energy)
    # gmx
    os.chdir(os.path.join(working_dir, "gmx"))

    GMX_set(CreateMDP=False, double_precision=True)
    GMX_run()

    gmx_energy=Load_energy(clean=False)
    gmx_forces=Load_forces(clean=False)
    print(gmx_energy)
    # Compare
    print("########################################")
    result_energy=Compare_energy(omm_energy[:,1:], gmx_energy[:,1:], isPrint=True)
    result_forces=Compare_forces(omm_forces[:,1:], gmx_forces[:,1:], isPrint=True)
    if not (result_energy and result_forces):
        raise AssertionError("Energies or forces do not match.")

In [4]:

class TestElectricField:
    """
    Test ElectricField
    """
    path = os.getcwd()

    def test_GlnBP(self):
        working_dir = os.path.join(self.path, "../data/EletricField/GlnBP_go_m3/")
        Compare_OMM_GMX(working_dir, epsilon_r = 15)  

In [5]:
test = TestElectricField()
test.test_GlnBP()

/home/ys/CommonUse/Martini/test/CTGoMartini/tests/api/../data/EletricField/GlnBP_go_m3/
[[      0.         -630565.75665008]]
[[      0.       -630565.756638]]
########################################
Energy Compare
Absolute error: 0.00001
Relative error: 0.00000
Energies match!
###Forces Compare###
Max absolute error: 1.93019
Max relative error: 0.17027
      Max allclose: 1.92856
Error: Forces do not match!


/home/ys/CommonUse/Martini/test/CTGoMartini/tests/api/function.py:325: RuntimeWarning: invalid value encountered in divide
  relative_force_error = np.linalg.norm(forces1 - forces2, axis=1) / average


AssertionError: Energies or forces do not match.